# Blend Mask (Cityscapes)

**(Esegui tutto)**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyyaml==5.1

# Cerchiamo la versione che porti il match perfetto tra le librerie
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]

from IPython.display import Image, clear_output

!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html

clear_output()

In [ ]:
# Cloning AdelaiDet
%cd /content
!git clone https://github.com/aim-uofa/AdelaiDet.git
%cd AdelaiDet/
!python setup.py build develop

clear_output()

In [ ]:
# Some basic setup:
import adet
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from adet.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
!wget -O blendmask_r101_3x.pth https://cloudstor.aarnet.edu.au/plus/s/e4fXrliAcMtyEBy/download

clear_output()

**Scelta del modello**

In [ ]:
model_BM_ResNet_101 = "configs/BlendMask/R_101_3x.yaml"
weight_BM_ResNet_101 = "blendmask_r101_3x.pth"

In [ ]:
!python -m pip install cityscapesscripts

clear_output()

In [ ]:
import functools
import json
import logging
import multiprocessing as mp
import numpy as np
import os
from itertools import chain
import pycocotools.mask as mask_util
from PIL import Image

from detectron2.structures import BoxMode
from detectron2.utils.comm import get_world_size
from detectron2.utils.file_io import PathManager
from detectron2.utils.logger import setup_logger

from cityscapesscripts.helpers.labels import labels
from cityscapesscripts.helpers.labels import id2label, name2label

try:
    import cv2
except ImportError:
    # OpenCV is an optional dependency at the moment
    pass


logger = logging.getLogger(__name__)


def _get_cityscapes_files(image_dir, gt_dir):
    files = []
    # scan through the directory
    cities = PathManager.ls(image_dir)
    logger.info(f"{len(cities)} cities found in '{image_dir}'.")
    for city in cities:
        city_img_dir = os.path.join(image_dir, city)
        city_gt_dir = os.path.join(gt_dir, city)
        for basename in PathManager.ls(city_img_dir):
            image_file = os.path.join(city_img_dir, basename)

            suffix = "leftImg8bit.png"
            assert basename.endswith(suffix), basename
            basename = basename[: -len(suffix)]

            instance_file = os.path.join(city_gt_dir, basename + "gtFine_instanceIds.png")
            label_file = os.path.join(city_gt_dir, basename + "gtFine_labelIds.png")
            json_file = os.path.join(city_gt_dir, basename + "gtFine_polygons.json")

            files.append((image_file, instance_file, label_file, json_file))
    assert len(files), "No images found in {}".format(image_dir)
    for f in files[0]:
        assert PathManager.isfile(f), f
    return files


def load_cityscapes_instances(image_dir, gt_dir, from_json=True, to_polygons=True):
    """
    Args:
        image_dir (str): path to the raw dataset. e.g., "~/cityscapes/leftImg8bit/train".
        gt_dir (str): path to the raw annotations. e.g., "~/cityscapes/gtFine/train".
        from_json (bool): whether to read annotations from the raw json file or the png files.
        to_polygons (bool): whether to represent the segmentation as polygons
            (COCO's format) instead of masks (cityscapes's format).

    Returns:
        list[dict]: a list of dicts in Detectron2 standard format. (See
        `Using Custom Datasets </tutorials/datasets.html>`_ )
    """
    if from_json:
        assert to_polygons, (
            "Cityscapes's json annotations are in polygon format. "
            "Converting to mask format is not supported now."
        )
    files = _get_cityscapes_files(image_dir, gt_dir)

    logger.info("Preprocessing cityscapes annotations ...")
    # This is still not fast: all workers will execute duplicate works and will
    # take up to 10m on a 8GPU server.
    pool = mp.Pool(processes=max(mp.cpu_count() // get_world_size() // 2, 4))

    ret = pool.map(
        functools.partial(_cityscapes_files_to_dict, from_json=from_json, to_polygons=to_polygons),
        files,
    )
    logger.info("Loaded {} images from {}".format(len(ret), image_dir))

    # Map cityscape ids to contiguous ids
    from cityscapesscripts.helpers.labels import labels

    labels = [l for l in labels if l.hasInstances and not l.ignoreInEval]
    dataset_id_to_contiguous_id = {l.id: idx for idx, l in enumerate(labels)}
    for dict_per_image in ret:
        for anno in dict_per_image["annotations"]:
            anno["category_id"] = dataset_id_to_contiguous_id[anno["category_id"]]
    return ret


def load_cityscapes_semantic(image_dir, gt_dir):
    """
    Args:
        image_dir (str): path to the raw dataset. e.g., "~/cityscapes/leftImg8bit/train".
        gt_dir (str): path to the raw annotations. e.g., "~/cityscapes/gtFine/train".

    Returns:
        list[dict]: a list of dict, each has "file_name" and
            "sem_seg_file_name".
    """
    ret = []
    # gt_dir is small and contain many small files. make sense to fetch to local first
    gt_dir = PathManager.get_local_path(gt_dir)
    for image_file, _, label_file, json_file in _get_cityscapes_files(image_dir, gt_dir):
        label_file = label_file.replace("labelIds", "labelTrainIds")

        with PathManager.open(json_file, "r") as f:
            jsonobj = json.load(f)
        ret.append(
            {
                "file_name": image_file,
                "sem_seg_file_name": label_file,
                "height": jsonobj["imgHeight"],
                "width": jsonobj["imgWidth"],
            }
        )
    assert len(ret), f"No images found in {image_dir}!"
    assert PathManager.isfile(
        ret[0]["sem_seg_file_name"]
    ), "Please generate labelTrainIds.png with cityscapesscripts/preparation/createTrainIdLabelImgs.py"  # noqa
    return ret


def _cityscapes_files_to_dict(files, from_json, to_polygons):
    """
    Parse cityscapes annotation files to a instance segmentation dataset dict.

    Args:
        files (tuple): consists of (image_file, instance_id_file, label_id_file, json_file)
        from_json (bool): whether to read annotations from the raw json file or the png files.
        to_polygons (bool): whether to represent the segmentation as polygons
            (COCO's format) instead of masks (cityscapes's format).

    Returns:
        A dict in Detectron2 Dataset format.
    """
    from cityscapesscripts.helpers.labels import id2label, name2label

    image_file, instance_id_file, _, json_file = files

    annos = []

    if from_json:
        from shapely.geometry import MultiPolygon, Polygon

        with PathManager.open(json_file, "r") as f:
            jsonobj = json.load(f)
        ret = {
            "file_name": image_file,
            "image_id": os.path.basename(image_file),
            "height": jsonobj["imgHeight"],
            "width": jsonobj["imgWidth"],
        }

        # `polygons_union` contains the union of all valid polygons.
        polygons_union = Polygon()

        # CityscapesScripts draw the polygons in sequential order
        # and each polygon *overwrites* existing ones. See
        # (https://github.com/mcordts/cityscapesScripts/blob/master/cityscapesscripts/preparation/json2instanceImg.py) # noqa
        # We use reverse order, and each polygon *avoids* early ones.
        # This will resolve the ploygon overlaps in the same way as CityscapesScripts.
        for obj in jsonobj["objects"][::-1]:
            if "deleted" in obj:  # cityscapes data format specific
                continue
            label_name = obj["label"]

            try:
                label = name2label[label_name]
            except KeyError:
                if label_name.endswith("group"):  # crowd area
                    label = name2label[label_name[: -len("group")]]
                else:
                    raise
            if label.id < 0:  # cityscapes data format
                continue

            # Cityscapes's raw annotations uses integer coordinates
            # Therefore +0.5 here
            poly_coord = np.asarray(obj["polygon"], dtype="f4") + 0.5
            # CityscapesScript uses PIL.ImageDraw.polygon to rasterize
            # polygons for evaluation. This function operates in integer space
            # and draws each pixel whose center falls into the polygon.
            # Therefore it draws a polygon which is 0.5 "fatter" in expectation.
            # We therefore dilate the input polygon by 0.5 as our input.
            poly = Polygon(poly_coord).buffer(0.5, resolution=4)

            if not label.hasInstances or label.ignoreInEval:
                # even if we won't store the polygon it still contributes to overlaps resolution
                polygons_union = polygons_union.union(poly)
                continue

            # Take non-overlapping part of the polygon
            poly_wo_overlaps = poly.difference(polygons_union)
            if poly_wo_overlaps.is_empty:
                continue
            polygons_union = polygons_union.union(poly)

            anno = {}
            anno["iscrowd"] = label_name.endswith("group")
            anno["category_id"] = label.id

            if isinstance(poly_wo_overlaps, Polygon):
                poly_list = [poly_wo_overlaps]
            elif isinstance(poly_wo_overlaps, MultiPolygon):
                poly_list = poly_wo_overlaps.geoms
            else:
                raise NotImplementedError("Unknown geometric structure {}".format(poly_wo_overlaps))

            poly_coord = []
            for poly_el in poly_list:
                # COCO API can work only with exterior boundaries now, hence we store only them.
                # TODO: store both exterior and interior boundaries once other parts of the
                # codebase support holes in polygons.
                poly_coord.append(list(chain(*poly_el.exterior.coords)))
            anno["segmentation"] = poly_coord
            (xmin, ymin, xmax, ymax) = poly_wo_overlaps.bounds

            anno["bbox"] = (xmin, ymin, xmax, ymax)
            anno["bbox_mode"] = BoxMode.XYXY_ABS

            annos.append(anno)
    else:
        # See also the official annotation parsing scripts at
        # https://github.com/mcordts/cityscapesScripts/blob/master/cityscapesscripts/evaluation/instances2dict.py  # noqa
        with PathManager.open(instance_id_file, "rb") as f:
            inst_image = np.asarray(Image.open(f), order="F")
        # ids < 24 are stuff labels (filtering them first is about 5% faster)
        flattened_ids = np.unique(inst_image[inst_image >= 24])

        ret = {
            "file_name": image_file,
            "image_id": os.path.basename(image_file),
            "height": inst_image.shape[0],
            "width": inst_image.shape[1],
        }

        for instance_id in flattened_ids:
            # For non-crowd annotations, instance_id // 1000 is the label_id
            # Crowd annotations have <1000 instance ids
            label_id = instance_id // 1000 if instance_id >= 1000 else instance_id
            label = id2label[label_id]
            if not label.hasInstances or label.ignoreInEval:
                continue

            anno = {}
            anno["iscrowd"] = instance_id < 1000
            anno["category_id"] = label.id

            mask = np.asarray(inst_image == instance_id, dtype=np.uint8, order="F")

            inds = np.nonzero(mask)
            ymin, ymax = inds[0].min(), inds[0].max()
            xmin, xmax = inds[1].min(), inds[1].max()
            anno["bbox"] = (xmin, ymin, xmax, ymax)
            if xmax <= xmin or ymax <= ymin:
                continue
            anno["bbox_mode"] = BoxMode.XYXY_ABS
            if to_polygons:
                # This conversion comes from D4809743 and D5171122,
                # when Mask-RCNN was first developed.
                contours = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[
                    -2
                ]
                polygons = [c.reshape(-1).tolist() for c in contours if len(c) >= 3]
                # opencv's can produce invalid polygons
                if len(polygons) == 0:
                    continue
                anno["segmentation"] = polygons
            else:
                anno["segmentation"] = mask_util.encode(mask[:, :, None])[0]
            annos.append(anno)
    ret["annotations"] = annos
    return ret

In [ ]:
cityscapes_classes = [k.name for k in labels if k.hasInstances and not k.ignoreInEval]

image_dir = "/content/drive/MyDrive/datasets/cityscapes/leftImg8bit/"
gt_dir = "/content/drive/MyDrive/datasets/cityscapes/gtFine/"

for d in ["train"]:
  DatasetCatalog.register("cityscapes_" + d, lambda x=image_dir+d, y=gt_dir+d: load_cityscapes_instances(x, y))
  MetadataCatalog.get("cityscapes_" + d).set(thing_classes=cityscapes_classes, evaluator_type="coco")

for d in ["val"]:
  DatasetCatalog.register("cityscapes_" + d, lambda x=image_dir+d, y=gt_dir+d: load_cityscapes_instances(x, y))
  MetadataCatalog.get("cityscapes_" + d).set(thing_classes=cityscapes_classes, evaluator_type="coco")

for d in ["test"]:
  DatasetCatalog.register("cityscapes_" + d, lambda x=image_dir+d, y=gt_dir+d: load_cityscapes_instances(x, y))
  MetadataCatalog.get("cityscapes_" + d).set(thing_classes=cityscapes_classes, evaluator_type="coco")

In [ ]:
from detectron2.engine import DefaultTrainer


cfg = get_cfg()
cfg.merge_from_file(model_BM_ResNet_101)
cfg.DATASETS.TRAIN = ("cityscapes_train",)
cfg.DATASETS.TEST = ()
cfg.MODEL.WEIGHTS = weight_BM_ResNet_101 


cfg.MODEL.BASIS_MODULE.NORM = "BN" # Not SyncBN because we have only 1 GPU and 1 machine, otherwise there is a problem with init groups of process
cfg.MODEL.BASIS_MODULE.NUM_CLASSES = 8 # Classi di Cityscapes
cfg.MODEL.FCOS.NUM_CLASSES = 8
cfg.MODEL.MEInst.NUM_CLASSES = 8
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8

cfg.MODEL.BASIS_MODULE.LOSS_ON = False


cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.SOLVER.IMS_PER_BATCH = 1
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 2000
cfg.SOLVER.STEPS = (1500,)




os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

WARNING [02/06 14:13:07 d2.config.compat]: Config 'configs/BlendMask/R_101_3x.yaml' has no VERSION. Assuming it to be compatible with latest v2.
[02/06 14:13:21 d2.engine.defaults]: Model:
BlendMask(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Skip loading parameter 'proposal_generator.fcos_head.cls_logits.weight' to the model due to incompatible shapes: (80, 256, 3, 3) in the checkpoint but (8, 256, 3, 3) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.fcos_head.cls_logits.bias' to the model due to incompatible shapes: (80,) in the checkpoint but (8,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
proposal_generator.fcos_head.cls_log

[02/06 14:19:28 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/usr/local/lib/python3.7/dist-p

[02/06 14:19:56 d2.utils.events]:  eta: 0:42:46  iter: 19  total_loss: 2.213  loss_mask: 0.288  loss_fcos_cls: 1.102  loss_fcos_loc: 0.204  loss_fcos_ctr: 0.6277  time: 1.3437  data_time: 0.0495  lr: 1.9981e-05  max_mem: 2092M
[02/06 14:20:21 d2.utils.events]:  eta: 0:42:23  iter: 39  total_loss: 2.188  loss_mask: 0.2884  loss_fcos_cls: 1.056  loss_fcos_loc: 0.1967  loss_fcos_ctr: 0.6171  time: 1.3100  data_time: 0.0045  lr: 3.9961e-05  max_mem: 2092M
[02/06 14:20:47 d2.utils.events]:  eta: 0:42:01  iter: 59  total_loss: 2.098  loss_mask: 0.2643  loss_fcos_cls: 1.023  loss_fcos_loc: 0.214  loss_fcos_ctr: 0.6277  time: 1.3013  data_time: 0.0052  lr: 5.9941e-05  max_mem: 2092M
[02/06 14:21:13 d2.utils.events]:  eta: 0:41:36  iter: 79  total_loss: 1.86  loss_mask: 0.2226  loss_fcos_cls: 0.8302  loss_fcos_loc: 0.1838  loss_fcos_ctr: 0.619  time: 1.2979  data_time: 0.0048  lr: 7.9921e-05  max_mem: 2092M
[02/06 14:21:39 d2.utils.events]:  eta: 0:41:12  iter: 99  total_loss: 1.743  loss_mask:

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("cityscapes_val", cfg, False, output_dir="./output")
val_loader = build_detection_test_loader(cfg, "cityscapes_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

WARNING [02/06 15:02:35 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[02/06 15:02:35 d2.evaluation.coco_evaluation]: Trying to convert 'cityscapes_val' to COCO format ...
[02/06 15:02:35 d2.data.datasets.coco]: Converting annotations of dataset 'cityscapes_val' to COCO format ...)
[02/06 15:03:50 d2.data.datasets.coco]: Converting dataset dicts into COCO format
[02/06 15:03:53 d2.data.datasets.coco]: Conversion finished, #images: 500, #annotations: 10657
[02/06 15:03:53 d2.data.datasets.coco]: Caching COCO format annotations at './output/cityscapes_val_coco_format.json' ...
[02/06 15:04:55 d2.data.build]: Distribution of instances among all 8 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|   person   | 3400         |   rider    | 544          |    car

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/usr/local/lib/python3.7/dist-p

[02/06 15:05:13 d2.evaluation.evaluator]: Inference done 11/500. Dataloading: 0.0012 s/iter. Inference: 0.6692 s/iter. Eval: 0.8641 s/iter. Total: 1.5345 s/iter. ETA=0:12:30
[02/06 15:05:19 d2.evaluation.evaluator]: Inference done 15/500. Dataloading: 0.0018 s/iter. Inference: 0.6656 s/iter. Eval: 0.8499 s/iter. Total: 1.5175 s/iter. ETA=0:12:15
[02/06 15:05:25 d2.evaluation.evaluator]: Inference done 19/500. Dataloading: 0.0018 s/iter. Inference: 0.6655 s/iter. Eval: 0.8540 s/iter. Total: 1.5216 s/iter. ETA=0:12:11
[02/06 15:05:31 d2.evaluation.evaluator]: Inference done 23/500. Dataloading: 0.0021 s/iter. Inference: 0.6634 s/iter. Eval: 0.8462 s/iter. Total: 1.5121 s/iter. ETA=0:12:01
[02/06 15:05:37 d2.evaluation.evaluator]: Inference done 27/500. Dataloading: 0.0023 s/iter. Inference: 0.6641 s/iter. Eval: 0.8512 s/iter. Total: 1.5180 s/iter. ETA=0:11:58
[02/06 15:05:44 d2.evaluation.evaluator]: Inference done 31/500. Dataloading: 0.0025 s/iter. Inference: 0.6651 s/iter. Eval: 0.852

**Stampa immagini**

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
model_config = open("cfg.yaml", "w")
model_config.write(str(cfg))
model_config.close()

In [ ]:
!python demo/demo.py \
    --config-file /content/AdelaiDet/cfg.yaml \
    --input /content/drive/MyDrive/datasets/cityscapes/leftImg8bit/test/bonn/bonn_000013_000019_leftImg8bit.png \
    --confidence-threshold 0.5 \
    --output outBM1.jpg \
    --opts MODEL.WEIGHTS /content/AdelaiDet/output/blendmask/R_101_3x/model_final.pth

[02/06 15:21:15 detectron2]: Arguments: Namespace(confidence_threshold=0.5, config_file='/content/AdelaiDet/cfg.yaml', input=['/content/drive/MyDrive/datasets/cityscapes/leftImg8bit/test/bonn/bonn_000013_000019_leftImg8bit.png'], opts=['MODEL.WEIGHTS', '/content/AdelaiDet/output/blendmask/R_101_3x/model_final.pth'], output='outBM1.jpg', video_input=None, webcam=False)
The checkpoint state_dict contains keys that are not used by the model:
  backbone.fpn_lateral3.bias
  backbone.fpn_output3.bias
  backbone.fpn_lateral4.bias
  backbone.fpn_output4.bias
  backbone.fpn_lateral5.bias
  backbone.fpn_output5.bias
  0% 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, roun

In [ ]:
out1 = cv2.imread("outBM1.jpg")
cv2_imshow(out1)